
## <ins>Filtrage basé sur le contenu</ins>

Dans le filtrage basé sur le contenu on utilise l'information qu'on connaît sur les intérêts des utilisateurs comme liaison pour les recommandations potentielles.

Supposons qu'on demande à Alice le type de livres qu'elle aime afin de savoir si elle serait intéressée par Le Hobbit. Avec ces informations, on commence par labeliser les utilisateurs (users) ainsi que les produits (items, dans ce cas les livres) par des variables connues, par exemple aventure et romance.

Si Alice n'aime pas l'aventure mais elle aime la romance, on peut représenter ses préférences commme le vecteur  (0,4),en supposant une notation entre  0−5.

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/raw/silver/processed_movies.csv')

movies.head()

,movieid,title,genres,year
0,1,Toy Story,"Adventure, Animation, Children, Comedy, Fantasy",1995
1,2,Jumanji,"Adventure, Children, Fantasy",1995
2,3,Grumpier Old Men,"Comedy, Romance",1995
3,4,Waiting to Exhale,"Comedy, Drama, Romance",1995
4,5,Father of the Bride Part II,Comedy,1995


Pour notre système de recommandation on utilisera la colonne 'genres' pour trouver ces mots et ainsi repérer les livres qui portent sur des sujets similaires.

Pour ce faire, on passera par la tokenization et vectorisation. Pour l’instant, tout ce qu'il faut savoir c’est que ce sont des processus fondamentaux en traitement du langage naturel (TLN) qui consistent à convertir les données textuelles en un format compréhensible et exploitable par les modèles d'apprentissage automatique.

La tokenization est le processus qui consiste à diviser un texte en unités individuelles (mots ou des sous-mots) appelées des "tokens".

La vectorisation consiste à représenter chaque "token" sous une forme numérique , généralement sous forme d'un vecteur de nombres. Ceci est nécessaire car la plupart des modèles d'apprentissage automatique requièrent une entrée numérique.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Créer un TfidfVectorizer et supprimer les mots vides
tfidf = TfidfVectorizer()
# Adapter et transformer les données en une matrice tfidf
matrice_tfidf = tfidf.fit_transform(movies['genres'])
# Afficher la forme de la matrice tfidf
print(matrice_tfidf.shape)

(27278, 24)


Le nombre de lignes (27278) correspond au nombre de films dans notre DataFrame. Quant au nombre de colonnes (24), ceci correspond au nombre de mots importants.

Ensuite on va calculer la similarité entre chaque livre.

Les fonctions cosine_similarity et euclidean_distances de la bibliothèque sklearn.metrics.pairwise font le calcul de la similarité cosinus et distance euclidienne, respectivement.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

sim_euclidienne = 1 / (1 +euclidean_distances(matrice_tfidf))
sim_cosinus = cosine_similarity(matrice_tfidf, matrice_tfidf)

In [5]:
indices = pd.Series(range(0,len(movies)), index=movies['title'])

In [7]:
from tabulate import tabulate

def recommandations(titre, mat_sim, num_recommandations=10):
    """Fonction qui  à partir des indices trouvés, renvoye les titres des films les plus similaires."""
    # récupérer dans idx l'indice associé au titre depuis la série indices
    idx = indices[titre]
    # garder dans une liste les scores de similarité correspondants à l'index du film cible
    score_sim = list(enumerate(mat_sim[idx]))
    #  trier les scores de similarité, trouver les plus similaires et récupérer ses indices
    score_sim = sorted(score_sim, key=lambda x: x[1], reverse=True)
    # Obtenir les scores des 10 films les plus similaires
    top_similair = score_sim[1:num_recommandations+1]
    # Obtenir les indices des films
    res = [(indices.index[idx], score) for idx, score in top_similair]
    # Renvoyer les titres des films les plus similaires
    return tabulate(res, headers=["Titre", "Score de similarité"], tablefmt="pretty")


In [8]:
print('RECOMMNDATION SIMILARITES EUCLIDIENNE')
print(recommandations('Toy Story', sim_euclidienne))

print('RECOMMNDATION SIMILARITES COSINUS')
print(recommandations('Toy Story', sim_cosinus))

print('RECOMMNDATION SIMILARITES EUCLIDIENNE')
print(recommandations("Jumanji", sim_euclidienne))
print('RECOMMNDATION SIMILARITES COSINUS')
print(recommandations("Jumanji", sim_cosinus))

RECOMMNDATION SIMILARITES EUCLIDIENNE
+--------------------------------------------------+---------------------+
|                      Titre                       | Score de similarité |
+--------------------------------------------------+---------------------+
|                       Antz                       |         1.0         |
|                   Toy Story 2                    |         1.0         |
|     Adventures of Rocky and Bullwinkle, The      |         1.0         |
|            Emperor's New Groove, The             |         1.0         |
|                  Monsters, Inc.                  |         1.0         |
| DuckTales: The Movie - Treasure of the Lost Lamp |         1.0         |
|                    Wild, The                     |         1.0         |
|                 Shrek the Third                  |         1.0         |
|             Tale of Despereaux, The              |         1.0         |
| Asterix and the Vikings (Astérix et les Vikings) |         1